Import Libraries

In [1]:
import pandas as pd
import numpy as np
#from sklearn import linear_model
#from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
#from sklearn.preprocessing import MinMaxScaler

Load Data Set

In [2]:
#Load Train Data
df_tr = pd.read_csv("E:\\TCD Assignments\\Machine Learning\\Kaggle\\TCD_Dataset.csv")
df_tr = df_tr.drop(['Instance','Wears Glasses','Hair Color','Body Height [cm]'],axis=1)

#Load Test Data
df_ts = pd.read_csv("E:\\TCD Assignments\\Machine Learning\\Kaggle\\Test_Data.csv")
df_ts = df_ts.drop(['Instance','Wears Glasses','Hair Color','Body Height [cm]'],axis=1)

Fill Missing Data

In [3]:
def fillMissingValue(X):
    d = {}
    for c in list(X.columns):
        if X[c].dtypes == 'object':
            d[c] = 'NA' #X[c].value_counts().index[0]
        else:
            d[c] = X[c].mean()
    return X.fillna(value=d)

In [4]:
df_tr = fillMissingValue(df_tr)
df_ts = fillMissingValue(df_ts)

Use target Encoding to Encode Categirocal Variables

In [5]:
def calc_smooth_mean(df,cat_var,tgt_var,sm=300):
    dict_smooth = dict()
    #dict_gmean = dict()
    for col in cat_var:
        g_mean = df[tgt_var].mean()
        
        #calculate the number of values and the mean of each group
        agg = df.groupby(col)[tgt_var].agg(['count','mean'])
        agg_count = agg['count']
        agg_mean = agg['mean']
        
        #calculate smooth_mean
        smooth = (agg_count * agg_mean + sm * agg_mean) / (agg_count + sm)
        #print(smooth)
        df[col] =  df[col].map(smooth)
        #dict_gmean[col] = g_mean
        dict_smooth[col] = smooth
    return df,dict_smooth

Call TargetEncoding Function

In [6]:
df_tr,test_smooth = calc_smooth_mean(df_tr,[x for x in df_tr.columns if df_tr[x].dtypes == 'object'],'Income in EUR')

Add meanvalues to test set

In [7]:
for c in [x for x in df_ts.columns if df_ts[x].dtypes == 'object']:
    df_ts.loc[:,c] = df_ts[c].map(test_smooth[c])

Train the model

In [8]:
#Changing dataframe to numpy also works with dataframe
X_train = df_tr.iloc[:,:-1].values
y_train =  df_tr.iloc[:,-1].values
#numpy dataframe for test data
X_test = df_ts.iloc[:,:-1]
X_test = fillMissingValue(X_test).values


rfr = RandomForestRegressor(n_estimators=250, criterion='mse', max_depth=10)
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=250,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [9]:
#Predict the outcome
y_pred = rfr.predict(X_test)

In [10]:
print(y_pred)

[ 36301.069358    12718.06391399  32961.17297609 ...  62613.96164612
 179678.10929783 179563.06500265]
